In [ ]:
# load data
# load nodes for enterprises
# save attributes for enterprises
# identify the legal entity and aliases
# identify edges betw enterprises as count of documents exchanged

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# A directed graph class that can store multiedges.
G = nx.MultiDiGraph()

In [2]:
legal_entities = pd.DataFrame({'ID':[1,2,3,4,5,6], 
                   'Name':['A', 'H', 'J', 'U', 'E', 'F'], 
                   'Identifier': ['12345678', '12345679', '12345680', '12345683', '12345682', '12345681'],
                   'Country': ['BE', 'BE', 'BE', 'BE', 'BE', 'BE']
}) 

clients = pd.DataFrame({'ID': [42,23,43,4,5,9], 
                   'Name':['A', 'H', 'J', 'U', 'E', 'F'], 
                   'VAT': ['BE12345678', 'BE12345679', 'BE12345680', 'BE12345683', 'BE12345682', 'BE12345681'],
                   'CBE': ['12345678', '12345679', '12345680', pd.np.nan,  pd.np.nan, '12345681' ],
                   'Street': ['SomeStr1','SomeStr2','SomeStr3','SomeStr4','SomeStr5','SomeStr6'],
                   'City': ['Brussels','Brussels','Brussels','Brussels','Brussels','Brussels'],     
                   'Country': ['BE', 'BE', 'BE', 'BE', 'BE', 'BE'],
                   'Email': ['felix.revert@gmail.com', 'jean.durand@gmail.com', 'j.custom@gmail.com', pd.np.nan, 'j.custom@gmail.com', pd.np.nan]    
})

debtors = pd.DataFrame({'ID':[1,2,3,4,5,6], 
                   'Name':['A', 'B', 'C', 'U', 'E', 'F'], 
                   'CBE': ['12345678', '12666679', '12567680', '12345681', '12345682', '12345683'],
                   'Country': ['BE', 'BE', 'BE', 'BE', 'BE', 'BE']}) 

creditors = pd.DataFrame({'ID':[1,2,3,4,5,6], 
                   'Name':['A', 'X', 'Y', 'U', 'E', 'F'], 
                   'CBE': ['12345678', '12345665', '12345443', '12345681', '12345683', '12345682'],
                   'Country': ['BE', 'BE', 'BE', 'BE', 'BE', 'BE']}) 

In [3]:
legal_entities['LegalEntity'] = legal_entities['Country'] + legal_entities['Identifier'] 
legal_entities

,Country,ID,Identifier,Name,LegalEntity
0,BE,1,12345678,A,BE12345678
1,BE,2,12345679,H,BE12345679
2,BE,3,12345680,J,BE12345680
3,BE,4,12345683,U,BE12345683
4,BE,5,12345682,E,BE12345682
5,BE,6,12345681,F,BE12345681


In [4]:
#2 Identify Clients (enterprises) linked to legal_entities nodes using intersection - merge
# clients[('LegalEntity'] = clients['Country'] + clients['CBE'] 
clients['LegalEntity'] = clients['Country'] + (clients['CBE'])
## fill in LegalEntity from BE VAT (exceptionally it works like this)
idx = clients[(clients['LegalEntity'].isnull()) & (clients['Country']=='BE') & (clients['VAT'].notnull())].index
clients.at[idx,'LegalEntity'] = clients.loc[idx]['VAT']
clients

,CBE,City,Country,Email,ID,Name,Street,VAT,LegalEntity
0,12345678,Brussels,BE,felix.revert@gmail.com,42,A,SomeStr1,BE12345678,BE12345678
1,12345679,Brussels,BE,jean.durand@gmail.com,23,H,SomeStr2,BE12345679,BE12345679
2,12345680,Brussels,BE,j.custom@gmail.com,43,J,SomeStr3,BE12345680,BE12345680
3,NaN,Brussels,BE,NaN,4,U,SomeStr4,BE12345683,BE12345683
4,NaN,Brussels,BE,j.custom@gmail.com,5,E,SomeStr5,BE12345682,BE12345682
5,12345681,Brussels,BE,NaN,9,F,SomeStr6,BE12345681,BE12345681


In [5]:
# match legal_entities to clients
clients['legal_entity_id'] = pd.merge(clients,legal_entities,left_on=['LegalEntity'], 
                                      right_on = ['LegalEntity'], how = 'inner').ID_y
clients['legal_entity_id'] = clients['legal_entity_id'].fillna(-1).astype(np.int64)
clients

,CBE,City,Country,Email,ID,Name,Street,VAT,LegalEntity,legal_entity_id
0,12345678,Brussels,BE,felix.revert@gmail.com,42,A,SomeStr1,BE12345678,BE12345678,1
1,12345679,Brussels,BE,jean.durand@gmail.com,23,H,SomeStr2,BE12345679,BE12345679,2
2,12345680,Brussels,BE,j.custom@gmail.com,43,J,SomeStr3,BE12345680,BE12345680,3
3,NaN,Brussels,BE,NaN,4,U,SomeStr4,BE12345683,BE12345683,4
4,NaN,Brussels,BE,j.custom@gmail.com,5,E,SomeStr5,BE12345682,BE12345682,5
5,12345681,Brussels,BE,NaN,9,F,SomeStr6,BE12345681,BE12345681,6


In [6]:
#Debtors
debtors['LegalEntity'] = debtors['Country'] + (debtors['CBE'])
# match legal_entities to clients
debtors['legal_entity_id'] = pd.merge(debtors,legal_entities,left_on=['LegalEntity'], 
                                      right_on = ['LegalEntity'], how = 'inner').ID_y
debtors['legal_entity_id']=debtors['legal_entity_id'].fillna(-1).astype(np.int64)
debtors

,CBE,Country,ID,Name,LegalEntity,legal_entity_id
0,12345678,BE,1,A,BE12345678,1
1,12666679,BE,2,B,BE12666679,6
2,12567680,BE,3,C,BE12567680,5
3,12345681,BE,4,U,BE12345681,4
4,12345682,BE,5,E,BE12345682,-1
5,12345683,BE,6,F,BE12345683,-1


In [7]:
#creditors
creditors['LegalEntity'] = creditors['Country'] + (creditors['CBE'])
# match legal_entities to clients
creditors['legal_entity_id'] = pd.merge(creditors,legal_entities,left_on=['LegalEntity'], 
                                      right_on = ['LegalEntity'], how = 'inner').ID_y
creditors['legal_entity_id']=creditors['legal_entity_id'].fillna(-1).astype(np.int64)
creditors

,CBE,Country,ID,Name,LegalEntity,legal_entity_id
0,12345678,BE,1,A,BE12345678,1
1,12345665,BE,2,X,BE12345665,6
2,12345443,BE,3,Y,BE12345443,4
3,12345681,BE,4,U,BE12345681,5
4,12345683,BE,5,E,BE12345683,-1
5,12345682,BE,6,F,BE12345682,-1


In [18]:
#1 add legal_entities as nodes
G.add_nodes_from(legal_entities['ID'], node_type = 'LegalEntity')

# add legal entity identifier attributes LegalEntity
attrs = legal_entities[['LegalEntity','ID']].set_index('ID').T.to_dict()
nx.set_node_attributes(G, attrs)

list(G.nodes(data=True))


[(1, {'LegalEntity': 'BE12345678', 'node_type': 'LegalEntity'}),
 (42, {}),
 (2, {'LegalEntity': 'BE12345679', 'node_type': 'LegalEntity'}),
 (23, {}),
 (3, {'LegalEntity': 'BE12345680', 'node_type': 'LegalEntity'}),
 (43, {}),
 (4, {'LegalEntity': 'BE12345683', 'node_type': 'LegalEntity'}),
 (5, {'LegalEntity': 'BE12345682', 'node_type': 'LegalEntity'}),
 (6, {'LegalEntity': 'BE12345681', 'node_type': 'LegalEntity'}),
 (9, {})]

In [60]:
# add enterprises nodes and edges linked to the legal entity
iter = clients[(clients['legal_entity_id']>=0)].index
for n in iter:
    legal_entity_id = clients.loc[n]['legal_entity_id']
    new_node_id = clients.loc[n]['ID'] 
    identifier = clients.loc[n]['LegalEntity']
    G.add_edge(legal_entity_id,new_node_id, Identifier = identifier)
    # add attributes to the new node
    
    attrs = clients.loc[n].to_dict()
   
   # nx.set_node_attributes(G.node[new_node_id], attrs)

    G.node[new_node_id]=attrs
    
#G = nx.from_pandas_edgelist(df=clients, source='legal_entity_id', target='ID')
#list(G.nodes)
# nx.get_node_attributes(G,'node_type')
list(G.edges(data=True))

AttributeError: 'dict' object has no attribute 'nodes'

In [47]:
# add enterprises nodes and edges linked to the legal entity


[(1, 42, {}), (2, 23, {}), (3, 43, {}), (4, 4, {}), (5, 5, {}), (6, 9, {})]

In [53]:
clients.loc[n].to_dict()

{'CBE': '12345678',
 'City': 'Brussels',
 'Country': 'BE',
 'Email': 'felix.revert@gmail.com',
 'ID': 42,
 'LegalEntity': 'BE12345678',
 'Name': 'A',
 'Street': 'SomeStr1',
 'VAT': 'BE12345678',
 'legal_entity_id': 1}

In [54]:
G.node[new_node_id]

{'node_type': 'Enterprise'}

In [59]:
clients.loc[n].to_dict() 

{'CBE': '12345678',
 'City': 'Brussels',
 'Country': 'BE',
 'Email': 'felix.revert@gmail.com',
 'ID': 42,
 'LegalEntity': 'BE12345678',
 'Name': 'A',
 'Street': 'SomeStr1',
 'VAT': 'BE12345678',
 'legal_entity_id': 1}